In [362]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import zipfile
import os

## DASK

В accounts.\*.csv найти `id`, для которого в столбце `amount` встречается наибольшее количество значений между 1000 и 1500. Выполнить задание с помощью Dask, распараллелив процесс обработки данных.

In [364]:
data_dir = f'{os.path.expanduser("~")}/Downloads/datasets/part_1/'

with zipfile.ZipFile(data_dir + 'data.zip') as archive:
    print(*archive.namelist(), sep='\n')
    archive.extractall(data_dir, None, None)

#parts = [delayed(pd.read_csv)(data_dir + file)
#            for file in archive.namelist() if file.endswith('.csv')]
#df = dd.from_delayed(parts)
df = dd.read_csv(data_dir + 'data/*.csv')

data/accounts.0.csv
data/accounts.1.csv
data/accounts.2.csv
data/accounts.3.csv
data/accounts.4.csv
data/accounts.5.csv
data/


In [358]:
df.assign(between = df['amount'].between(1000, 1500, 'neither')) \
            .groupby(['id'])['between'] \
            .sum() \
            .idxmax() \
            .compute()

485

В accounts.\*.csv найти `id`, для которого в столбце `amount` встречается наибольшее количество значений кратных трем. Выполнить задание с помощью Dask, распараллелив процесс обработки данных.

In [359]:
df.assign(three = ~df['amount'].apply(lambda x: x%3 and 1, meta=('amount', 'int')).astype(bool)) \
            .groupby(['id'])['three'] \
            .sum() \
            .idxmax() \
            .compute()

366

## Numpy

Дан массив A. Построить массив B той же размерности, состоящий из элементов, являющихся суммой минимального элемнета массива A по соответствующей строке и минимального элемента по соответствующему столбцу.

In [205]:
A = np.array([[ 8,  4, 16,  5,  9],
              [ 3, 16, 13,  8, 10],
              [ 0,  9, 18, 17,  5],
              [16,  2,  6,  0, 10],
              [18, 13,  9, 17,  8]])

In [361]:
ndim = len(A)

B = np.tile(A.min(axis=1), (ndim,1)).T + np.tile(A.min(axis=0), (ndim,1))
print(B)

[[ 4  6 10  4  9]
 [ 3  5  9  3  8]
 [ 0  2  6  0  5]
 [ 0  2  6  0  5]
 [ 8 10 14  8 13]]


In [347]:
B = np.add.reduce(np.meshgrid(*[A.min(axis=1), A.min(axis=0)])).T
print(B)

[[ 4  6 10  4  9]
 [ 3  5  9  3  8]
 [ 0  2  6  0  5]
 [ 0  2  6  0  5]
 [ 8 10 14  8 13]]


Задать два двухмерных массива размерности 5x15, состоящих из случайных целых чисел в интервале от -5 до 5. Если значение в первом массиве имеет тот же знак, что соответствующее значение во втором, то сложить эти значения.

In [332]:
A1 = np.random.randint(-5,5,size=(5,15))
A2 = np.random.randint(-5,5,size=(5,15))

In [333]:
print(A1); print(A2)

[[-1  2  3  1 -4  1  2  1  2 -4  1 -5  2  4  2]
 [-5  1  3  2  0 -4  2 -2 -4  2  0 -5  3 -4  4]
 [-2 -4 -3 -1  4  1  2 -3  1  0  1 -3 -5  1  3]
 [ 2  3 -2  1  3 -5  3 -5  0 -2 -2  2 -5 -5  0]
 [ 4  0 -4 -5  4  3 -1 -2  2 -1  4  2 -3  3  1]]
[[ 0 -4 -5  0 -2 -1  1 -4  0  2  1 -4  4  1  2]
 [-4  1 -1 -1 -3  1 -3 -5 -5 -2  4  2 -5 -5  0]
 [ 3  1 -2  2  1  2 -2 -5 -2 -3 -2  2 -3 -2 -1]
 [-4 -1  2 -1 -2 -3 -1 -2 -1  4  0  4  4 -2  2]
 [ 3  4  4  4  3 -1 -1  3  4  3 -3  2 -5 -4  0]]


In [338]:
print(np.where(np.sign(A1) == np.sign(A2), np.add(A1, A2), A1))

[[-1  2  3  1 -6  1  3  1  2 -4  2 -9  6  5  4]
 [-9  2  3  2  0 -4  2 -7 -9  2  0 -5  3 -9  4]
 [-2 -4 -5 -1  5  3  2 -8  1  0  1 -3 -8  1  3]
 [ 2  3 -2  1  3 -8  3 -7  0 -2 -2  6 -5 -7  0]
 [ 7  0 -4 -5  7  3 -2 -2  6 -1  4  4 -8  3  1]]
